In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data (replace this with your actual data)
data = {
    'item_id': [1, 2, 3, 4],
    'title': [['1','2'], ['2','3'], ['3','2'], ['4','5']]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the vectorizer on the item titles
X = vectorizer.fit_transform(df['title'])

# Calculate the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(X, X)

# Convert the similarity matrix to a DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=df['item_id'], columns=df['item_id'])

print(cosine_sim_df)

In [9]:
from itertools import chain
tokenized_list = list(chain(*df['title']))

In [50]:
import numpy as np
from sklearn.metrics import pairwise_distances
# Sample data (replace this with your actual data)
data = {
    'item_id': [1, 2, 3, 4],
    'tags': [[2, 8, 4], [2, 8, 4], [7, 5, 2], [3, 6, 9]]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Convert tags to a numpy array
tag_vectors = np.array(df['tags'].tolist())

# Calculate the cosine similarity matrix
cosine_sim_matrix = pairwise_distances(tag_vectors, metric='jaccard')

# Convert the similarity matrix to a DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=df['item_id'], columns=df['item_id'])

print(cosine_sim_df)




item_id    1    2    3    4
item_id                    
1        0.0  0.0  0.0  0.0
2        0.0  0.0  0.0  0.0
3        0.0  0.0  0.0  0.0
4        0.0  0.0  0.0  0.0


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:1875: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [20]:
import numpy as np
from sklearn.metrics import pairwise_distances

# Sample data: Replace this with your actual data
num_items = 1000000
num_tags = 50
data = np.random.randint(2, size=(num_items, num_tags))

In [ ]:


# Convert tags to binary vectors
tag_matrix = pd.get_dummies(tag_vectors, columns=range(1, 1001), prefix='tag')

# Calculate Jaccard similarity
jaccard_similarity_matrix = pairwise_distances(tag_matrix, metric='jaccard')

In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score
from multiprocessing import Pool
from scipy.sparse import coo_matrix, vstack

# Assuming 'df' is your dataframe with descriptions and tag lists
# 'tags' is a list of list of integer tags for each item
#df['tags'] = tags

# Convert tag lists to sets for faster intersection calculation
df['tag_sets'] = df['tags'].apply(set)
df['title_sets'] = df['title'].apply(set)

# Calculate Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    #union_size = len(set1) + len(set2) - intersection_size
    return intersection_size #/ union_size if union_size > 0 else 0.0

# Function to compute similarities for a range of items
def compute_similarities_range(start, end):
    tags_tags_similarities = np.zeros((end - start, num_items))
    tags_title_similarities = np.zeros((end - start, num_items))
    title_title_similarities = np.zeros((end - start, num_items))
    title_tags_similarities = np.zeros((end - start, num_items))
    for i in range(start, end):
        tags_tags_similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
        tags_title_similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['title_sets'][i], x))
        title_tags_similarities[i - start] = df['title_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
        title_title_similarities[i - start] = df['title_sets'].apply(lambda x: jaccard_similarity(df['title_sets'][i], x))
        similarities = tags_tags_similarities,tags_title_similarities,title_tags_similarities, title_title_similarities
        #similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
        #similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
    return similarities

num_items = len(df)

# Define the number of parallel processes based on your system's capabilities
num_processes = 4

# Split the computation into chunks for parallel processing
chunk_size = num_items // num_processes
chunks = [(i * chunk_size, (i + 1) * chunk_size) for i in range(num_processes - 1)]
chunks.append(((num_processes - 1) * chunk_size, num_items))

# Initialize a pool of processes
with Pool(num_processes) as pool:
    results = pool.starmap(compute_similarities_range, chunks)

# Combine the results from different processes
jaccard_similarity_matrix = np.vstack(results)

# Clean up temporary columns
df.drop(['tags', 'tag_sets'], axis=1, inplace=True)

# The jaccard_similarity_matrix will contain similarity values
# where jaccard_similarity_matrix[i, j] represents similarity between item i and item j


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score
from multiprocessing import Pool
from scipy.sparse import coo_matrix, vstack

# Assuming 'df' is your dataframe with descriptions and tag lists
# 'tags' is a list of list of integer tags for each item
#df['tags'] = tags

# Convert tag lists to sets for faster intersection calculation
df['tag_sets'] = df['tags'].apply(set)
df['title_sets'] = df['title'].apply(set)

# Calculate Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    #union_size = len(set1) + len(set2) - intersection_size
    return intersection_size #/ union_size if union_size > 0 else 0.0

# Function to compute similarities for a range of items
def compute_similarities_range(start, end):
    jobroles_tags_similarities = np.zeros((end - start, num_items))
    jobroles_title_similarities = np.zeros((end - start, num_items))
    title_title_similarities = np.zeros((end - start, num_items))
    title_tags_similarities = np.zeros((end - start, num_items))
    for i in range(start, end):
        tags_tags_similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
        tags_title_similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['title_sets'][i], x))
        title_tags_similarities[i - start] = df['title_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
        title_title_similarities[i - start] = df['title_sets'].apply(lambda x: jaccard_similarity(df['title_sets'][i], x))
        similarities = tags_tags_similarities,tags_title_similarities,title_tags_similarities, title_title_similarities
        #similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
        #similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
    return similarities

num_items = len(df)

# Define the number of parallel processes based on your system's capabilities
num_processes = 4

# Split the computation into chunks for parallel processing
chunk_size = num_items // num_processes
chunks = [(i * chunk_size, (i + 1) * chunk_size) for i in range(num_processes - 1)]
chunks.append(((num_processes - 1) * chunk_size, num_items))

# Initialize a pool of processes
with Pool(num_processes) as pool:
    results = pool.starmap(compute_similarities_range, chunks)

# Combine the results from different processes
jaccard_similarity_matrix = np.vstack(results)

# Clean up temporary columns
df.drop(['tags', 'tag_sets'], axis=1, inplace=True)

# The jaccard_similarity_matrix will contain similarity values
# where jaccard_similarity_matrix[i, j] represents similarity between item i and item j


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score
from multiprocessing import Pool
from scipy.sparse import coo_matrix, vstack

# Assuming 'df' is your dataframe with descriptions and tag lists
# 'tags' is a list of list of integer tags for each item
#df['tags'] = tags

# Convert tag lists to sets for faster intersection calculation
df['tag_sets'] = df['tags'].apply(set)
df['title_sets'] = df['title'].apply(set)

# Calculate Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    #union_size = len(set1) + len(set2) - intersection_size
    return intersection_size #/ union_size if union_size > 0 else 0.0

# Function to compute similarities for a range of items
def compute_similarities_range(start, end):
    tags_tags_similarities = np.zeros((end - start, num_items))
    tags_title_similarities = np.zeros((end - start, num_items))
    title_title_similarities = np.zeros((end - start, num_items))
    title_tags_similarities = np.zeros((end - start, num_items))
    for i in range(start, end):
        jobroles_tags_similarities[i - start] = items['tag_sets'].apply(lambda x: jaccard_similarity(assigned_users['jobroles_sets'][i], x))
        jobroles_title_similarities[i - start] = items['title_sets'].apply(lambda x: jaccard_similarity(assigned_users['jobroles_sets'][i], x))
        user_item_similarities = jobroles_tags_similarities,jobroles_title_similarities
        #similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
        #similarities[i - start] = df['tag_sets'].apply(lambda x: jaccard_similarity(df['tag_sets'][i], x))
    return user_item_similarities

num_items = len(df)

# Define the number of parallel processes based on your system's capabilities
num_processes = 4

# Split the computation into chunks for parallel processing
chunk_size = num_users // num_processes
chunks = [(i * chunk_size, (i + 1) * chunk_size) for i in range(num_processes - 1)]
chunks.append(((num_processes - 1) * chunk_size, num_users))

# Initialize a pool of processes
with Pool(num_processes) as pool:
    results = pool.starmap(compute_similarities_range, chunks)

# Combine the results from different processes
jaccard_similarity_matrix = np.vstack(results)

# Clean up temporary columns
df.drop(['tags', 'tag_sets'], axis=1, inplace=True)

# The jaccard_similarity_matrix will contain similarity values
# where jaccard_similarity_matrix[i, j] represents similarity between item i and item j


Features